## Install the required libraries

In [1]:

%pip install google-genai

%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Load API Keys + environment variables from .env file

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

def load_environment():
    """Load environment variables from multiple possible locations"""
    possible_paths = [
        Path('.env'),  # Current directory
        Path('../.env'),  # One level up
        Path('../../frontend/.env'),  # Frontend directory
        Path('../../../.env'),  # Project root
        Path('../../.env'),  # Project root alternative
    ]
    
    for path in possible_paths:
        if path.exists():
            print(f"Loading .env from: {path.absolute()}")
            load_dotenv(path)
            api_key = os.getenv("GEMINI_API_KEY")
            if api_key:
                print("✓ API key loaded successfully")
                return True
            
    print("⚠ No valid .env file found with API key!")
    print("\nTry creating .env file in one of these locations:")
    for path in possible_paths:
        print(f"- {path.absolute()}")
    return False

# Load environment variables
if not load_environment():
    raise EnvironmentError("Failed to load GEMINI_API_KEY from .env file")

# Now use the API key
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

Loading .env from: c:\UAL\Data-Driven Full Stack\CineMind\cinemind\backend\app\..\..\..\.env
✓ API key loaded successfully


## Test Gemini Model

In [5]:
from google import genai

# Initialize the client with your API key
client = genai.Client(api_key = GEMINI_API_KEY)

# Generate content using the Gemini model
try:
    response = client.models.generate_content(
        model="gemini-2.0-flash-lite",  # Replace with the specific model name
        contents="Explain how AI works",
    )
    print("Generated Content:")
    print(response.text)
except Exception as e:
    print(f"Error: {e}")

Generated Content:
AI, or Artificial Intelligence, is a broad field encompassing the creation of intelligent agents – systems that can reason, learn, and act autonomously.  It's not a single thing, but rather a collection of techniques and approaches.  Here's a breakdown of how AI generally works, covering key concepts and processes:

**1. Core Principles:**

*   **Mimicking Human Intelligence:**  The fundamental goal of AI is to create systems that can perform tasks that typically require human intelligence, such as:
    *   **Learning:**  Acquiring knowledge and skills from data.
    *   **Reasoning:**  Using logic and inference to draw conclusions.
    *   **Problem-solving:**  Finding solutions to complex issues.
    *   **Perception:**  Interpreting sensory input (sight, sound, etc.).
    *   **Natural Language Understanding (NLU):**  Comprehending and processing human language.
    *   **Planning:**  Developing strategies to achieve goals.
    *   **Decision-making:** Choosing th

In [ ]:
from google import genai
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from app import Movie

# Initialize the Google GenAI client
client = genai.Client(api_key=GEMINI_API_KEY)

# Connect to the database
db_path = os.path.abspath(os.path.join(os.getcwd(), '../models/cinemind.db'))
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file not found at: {db_path}")

engine = create_engine(f'sqlite:///{db_path}')
Session = sessionmaker(bind=engine)
session = Session()

# Fetch movies with reviews from the database
movies = session.query(Movie).filter(Movie.reviews.isnot(None)).limit(2).all()

# Analyze overall sentiment for all reviews of each movie
for movie in movies:
    try:
        review_texts = movie.reviews  # Access the reviews field directly
        response = client.models.generate_content(
            model="gemini-2.0-flash-lite",  # Replace with the specific model name
            contents="Analyze the sentiment of the following movie reviews and classify it as Positive, Neutral, or Negative. "
                "Provide a plain-text summary of the reviews without any titles, names of people, headings, or special formatting. "
                "Focus on the overall sentiment and key points expressed in the reviews as a whole and provide and overall summary:\n\n"
                f"{movie.reviews}",
        )
        print(f"Movie: {movie.title}")
        print(f"Overall Reviews: {review_texts}")
        print(f"Overall Sentiment: {response.text.strip()}")
        print("\n---\n")
    except Exception as e:
        print(f"Error analyzing overall sentiment for movie '{movie.title}': {e}")

Movie: Star Wars
Overall Reviews: Author: Cat Ellington
(As I'm writing this review, Darth Vader's theme music begins to build in my mind...)

Well, it actually has a title, what the Darth Vader theme. And that title is "The Imperial March", composed by the great John Williams, whom, as many of you may already know, also composed the theme music for "Jaws" - that legendary score simply titled, "Main Title (Theme From Jaws)".

Now, with that lil' bit of trivia aside, let us procede with the fabled film currently under review: Star Wars. It had been at a drive-in theater in some small Illinois town or other where my mother, my older brother, and I had spent our weekly "Movie Date Night" watching this George Lucas directed cult masterpiece from our car in the parking lot. On the huge outdoor screen, the film appeared to be a silent one, but thanks to an old wire-attached speaker, we were able to hear both the character dialogue and soundtrack loud and clear. We even had ourselves a carful

## Analyze Sentiment of Movie Reviews

In [ ]:
movie_id = 12  # Replace with the desired movie ID
response = client.models.generate_content(
    model="gemini-2.0-flash-lite",
    contents=f"Analyze the overall sentiment of these reviews: '{movies[1].reviews}' and classify it as Positive, Neutral, or Negative. Provide a summary."
)
print(f"Sentiment Analysis for Movie ID {movie_id}:")
print(response.text.strip())